In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize_2022 import MeasurementOptimizer, DataProcess 
import matplotlib.pyplot as plt
from itertools import permutations, product
from scipy.sparse import coo_matrix
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxModel
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxBlock

## Input Setting

### All measurements and index

- ads.gas_inlet.F (0)
- ads.gas_outlet.F (1)
- ads.gas_outlet.T (2)
- ads.gas_outlet.z (3)
- des.gas_inlet.F (4)
- des.gas_outlet.F (5)
- des.gas_outlet.T (6)
- des.gas_outlet.z (7)
- ads.T19 (8)
- ads.T23 (9)
- ads.T28 (10)
- ads.z19 (11)
- ads.z23 (12)
- ads.z28 (13)

One-time cost ones: 0,1,2, 4,5,6, 8,9,10

Time-cost ones: 3,7,11,12,13

In [2]:
all_names = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 
             'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 
             'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 
             'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 
             'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']

In [3]:
dataObject = DataProcess()

# static measurement jacobian
static_ind = [0,1,2,4,5,6,8,9,10]
static_Nt = 110
static_jaco = dataObject.read_jaco('./RotaryBed/Q'+str(static_Nt)+'_scale.csv')
num_static = len(static_ind)

static_Q = []
for i in static_ind:
    static_Q.append(dataObject.split_jaco(static_jaco, i,static_Nt))

    
    
dynamic_ind = [3,7,11,12,13]
dynamic_Nt = 110
num_dynamic = len(dynamic_ind)
dynamic_jaco = dataObject.read_jaco('./RotaryBed/Q'+str(dynamic_Nt)+'_scale.csv')

dynamic_Q = []
for i in dynamic_ind:
    dynamic_Q.extend(dataObject.split_jaco(dynamic_jaco, i, dynamic_Nt))
    
print("static Q shape:", np.shape(static_Q))
print("dynamic Q shape:", np.shape(dynamic_Q))

jacobian shape: (1540, 5)
jacobian shape: (1540, 5)
static Q shape: (9, 110, 5)
dynamic Q shape: (550, 5)


In [4]:
static_name = [all_names[i] for i in static_ind]
dynamic_name = [all_names[i] for i in dynamic_ind]

num_param = 5
cost = None

num_total = num_static + num_dynamic*dynamic_Nt

In [5]:
calculator = MeasurementOptimizer(static_Q, dynamic_Q, static_Nt, dynamic_Nt, num_param, cost, verbose=True)

fim_expect = calculator.fim_computation()

print(np.shape(calculator.fim_collection))

(312481, 5, 5)


In [99]:
def check_fim(idx):
    fim = np.asarray(calculator.Q[idx]).T@np.asarray(calculator.Q[idx])
    print(fim)
    print("det:", np.linalg.det(fim))
    print("trace:", np.trace(fim))
    
check_fim(3)
print(calculator.total_no_measure)

[[ 2.91849642e+00 -3.49765975e-01  2.62707152e+00 -1.43627241e+01
   1.52316519e+00]
 [-3.49765975e-01  2.93872975e-01 -8.09388225e-01  6.22048398e+00
  -6.23235591e-01]
 [ 2.62707152e+00 -8.09388225e-01  7.33868134e+00 -5.80459197e+01
   2.30524213e+00]
 [-1.43627241e+01  6.22048398e+00 -5.80459197e+01  5.16864298e+02
  -1.53535661e+01]
 [ 1.52316519e+00 -6.23235591e-01  2.30524213e+00 -1.53535661e+01
   3.53482513e+00]]
det: 212.73672822446898
trace: 530.9501736586553
559


### Set up grey-box 

In [7]:
from greybox_generalize import LogDetModel

In [8]:
def build_model_external(m):
    ex_model = LogDetModel(num_para=5)
    m.egb = ExternalGreyBoxBlock()
    m.egb.set_external_model(ex_model)

### Reform the inputs

In [9]:
cost = [1000, #ads.gas_inlet.F (0)
        1000, #ads.gas_outlet.F (1)
         500, #ads.gas_outlet.T (2)
        1000, #des.gas_inlet.F (4)
        1000, #des.gas_outlet.F (5)
         500, #des.gas_outlet.T (6)
         1000, #ads.T19 (8)
         1000, #ads.T23 (9)
         1000 #ads.T28 (10)
       ]

for i in range(dynamic_Nt*num_dynamic):
    cost.append(100)

install_cost_dynamic = [100, 100, 500, 500, 500]

In [77]:
#print(np.shape(ans_y))

init_y = np.random.rand(num_total-num_static)

update_y = [1]*num_static 

for i in init_y:
    if i > 0.5: 
        update_y.append(1)
    else:
        update_y.append(0)

init_cov_y = [[0]*num_total for i in range(num_total)]

for j in range(num_total):
    for k in range(num_total):
        if j==k:
            init_cov_y[j][k] = update_y[k]
            
        else:
            init_cov_y[j][k] = min(update_y[k], update_y[j])

fim_prior = np.identity((5))
print(fim_prior)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [78]:
def compute_FIM(numRes, FIMSet, cost_list, mixed_integer=False,  obj="A", 
                num_fixed = 9, num_dynamic=5, num_dynamic_t = 100,
                num_dynamic_t_name = None, discretize_time = None, end_time=220,
                fix=False, sparse=False, manual_number=20, budget=100,
               dynamic_install_cost = None, each_manual_number=None):
    
    m = pyo.ConcreteModel()
    
    m.NumRes = pyo.Set(initialize=range(numRes))
    m.DimFIM = pyo.Set(initialize=range(len(FIMSet[0])))
    # dynamic measurements num 
    m.DimDynamic = pyo.Set(initialize=range(num_fixed, num_dynamic+num_fixed))
    m.DimDynamic_t = pyo.Set(initialize=range(num_dynamic_t))
    
    dynamic_time = {}
    for i in range(num_dynamic_t):
        dynamic_time[i] = num_dynamic_t_name[i]
        
    print(dynamic_time)
    
    # set up y and cov y 
    def identity(m, a, b):
        return 1 if a==b else 0
    
    def initialize(m, a, b):
        return init_cov_y[a][b]
    
    def NumReshalf_init(m):
        return ((a,b) for a in m.NumRes for b in range(a, numRes))
    
    def DimFIMhalf_init(m):
        return ((a,b) for a in m.DimFIM for b in range(a, len(FIMSet[0])))
    
    m.NumRes_half = pyo.Set(dimen=2, initialize=NumReshalf_init)
    m.DimFIM_half = pyo.Set(dimen=2, initialize=DimFIMhalf_init)

    if mixed_integer:
        if sparse:
            m.cov_y = pyo.Var(m.NumRes_half, initialize=initialize, bounds=(0,1), within=pyo.Binary)
        else:
            m.cov_y = pyo.Var(m.NumRes, m.NumRes, initialize=initialize, bounds=(0,1), within=pyo.Binary)
    else:
        if sparse:
            m.cov_y = pyo.Var(m.NumRes_half, initialize=initialize, bounds=(0,1), within=pyo.NonNegativeReals)
        else:
            m.cov_y = pyo.Var(m.NumRes, m.NumRes, initialize=initialize, bounds=(0,1), within=pyo.NonNegativeReals)
    
    if fix:
        m.cov_y.fix()
        
    def init_fim(m, p, q):
        return fim_prior[p,q]   
    
    # set up FIM
    if sparse:
        m.TotalFIM = pyo.Var(m.DimFIM_half, initialize=init_fim)
    else:
        m.TotalFIM = pyo.Var(m.DimFIM, m.DimFIM, initialize=init_fim)

    # compute FIM
    def eval_fim(m, a, b):
        if a <= b: 
            summi = 0 
            for i in range(numRes):
                for j in range(numRes):
                    if j>=i:
                        summi += m.cov_y[i,j]*FIMSet[i*numRes+j][a][b]
                    else:
                        summi += m.cov_y[j,i]*FIMSet[i*numRes+j][a][b]
            return m.TotalFIM[a,b] == summi
        else:
            return m.TotalFIM[a,b] == m.TotalFIM[b,a]
    
    m.TotalDynamic = pyo.Var(initialize=1)
    
    def total_dynamic(m):
        return m.TotalDynamic==sum(m.cov_y[i,i] for i in range(num_fixed, numRes))
    
    m.manual = pyo.Constraint(rule=total_dynamic)
    #m.manual = pyo.Expression(expr=sum(m.cov_y[i,i] for i in range(num_fixed, numRes)))
            
    # y and conv y constraints
    def y_covy1(m, a, b):
        if b > a:
            return m.cov_y[a, b] <= m.cov_y[a, a]
        else:
            return pyo.Constraint.Skip
        
    def y_covy2(m, a, b):
        if b > a:
            return m.cov_y[a, b] <= m.cov_y[b, b]
        else:
            return pyo.Constraint.Skip
        
    def y_covy3(m, a, b):
        if b>a:
            return m.cov_y[a, b] >= m.cov_y[a, a] + m.cov_y[b, b] - 1
        else:
            return pyo.Constraint.Skip
        
    def y_covy4(m, a, b):
        if a>b:
            return m.cov_y[a, b] == m.cov_y[a, a]*m.cov_y[b,b]
        else:
            return pyo.Constraint.Skip
        
    def symmetry(m,a,b):
        if a>b:
            return m.cov_y[a,b] == m.cov_y[b,a]
        else:
            return pyo.Constraint.Skip
    
    def cost_compute(m):
        cost = sum(m.cov_y[i,i]*cost_list[i] for i in range(numRes))+sum(m.if_install_dynamic[j]*dynamic_install_cost[j-num_fixed] for j in m.DimDynamic)
        return m.cost == cost

    def cost_limit(m):
        return m.cost <= budget
    
    def total_dynamic_con(m):
        return m.TotalDynamic<=manual_number
    
    def dynamic_fix_yd(m,i,j):
        # map measurement index i to its dynamic_flatten index
        start = num_fixed + (i-num_fixed)*num_dynamic_t+j
        return m.if_install_dynamic[i] >= m.cov_y[j,j]
    
    def dynamic_fix_yd_con2(m,i):
        start = num_fixed + (i-num_fixed)*num_dynamic_t
        end = num_fixed + (i-num_fixed+1)*num_dynamic_t
        return m.if_install_dynamic[i] <= sum(m.cov_y[j,j] for j in range(start, end))
    
    # set up Design criterion
    def ComputeTrace(m):
        sum_x = sum(m.TotalFIM[j,j] for j in m.DimFIM)
        return sum_x
    
    
    # add constraints
    if sparse:
        m.TotalFIM_con = pyo.Constraint(m.DimFIM_half, rule=eval_fim)
    else:
        m.TotalFIM_con = pyo.Constraint(m.DimFIM, m.DimFIM, rule=eval_fim)
    
    if not fix:
        
        if mixed_integer and not sparse:
            m.sym = pyo.Constraint(m.NumRes, m.NumRes, rule=symmetry)
        
        m.cov1 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy1)
        m.cov2 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy2)
        m.cov3 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy3)
            
        m.con_manual = pyo.Constraint(rule=total_dynamic_con)
        
        # each manual number smaller than 5 
        if each_manual_number:
            
            for i in range(num_dynamic):
                def dynamic_manual_num(m):
                    start = num_fixed + i*num_dynamic_t
                    end = num_fixed + (i+1)*num_dynamic_t
                    cost = sum(m.cov_y[j,j] for j in range(start, end))
                    return cost <= each_manual_number
                
                con_name = "con"+str(i)
                m.add_component(con_name, pyo.Constraint(expr=dynamic_manual_num))
                
        if discretize_time:
            
            for i in m.DimDynamic:
                for t in range(num_dynamic_t):
                    # end time is an open end of the region, so another constraint needs to be added to include end_time
                    #if dynamic_time[t]+discretize_time <= end_time+0.1*discretize_time:       
                                              
                    def discretizer(m):
                        sumi = 0

                        count = 0 
                        while (count+t<num_dynamic_t) and (dynamic_time[count+t]-dynamic_time[t])<discretize_time:
                            surro_idx = num_fixed + (i-num_fixed)*num_dynamic_t + t + count
                            sumi += m.cov_y[surro_idx, surro_idx]
                            count += 1 

                        return sumi <= 1 

                        #return sum(cov_y[j,j] for j in disc_ind) <= 1 

                    con_name="con_discreti_"+str(i)+str(t)
                    m.add_component(con_name, pyo.Constraint(expr=discretizer))
                    
        # dynamic-cost measurements installaction cost 
        m.if_install_dynamic = pyo.Var(m.DimDynamic, initialize=0, bounds=(0,1.01))
        m.dynamic_cost = pyo.Constraint(m.DimDynamic, m.DimDynamic_t, rule=dynamic_fix_yd)
        m.dynamic_con2 = pyo.Constraint(m.DimDynamic, rule=dynamic_fix_yd_con2)
                
        m.cost = pyo.Var(initialize=budget)
        m.cost_compute = pyo.Constraint(rule=cost_compute)
        m.budget_limit = pyo.Constraint(rule=cost_limit)
     
    # set objective 
    if obj == "A":
        m.Obj = pyo.Objective(rule=ComputeTrace, sense=pyo.maximize)    
    elif obj =="D":
        
        def _model_i(b):
            build_model_external(b)
        m.my_block = pyo.Block(rule=_model_i)
        
        for i in m.DimFIM:
            for j in range(i, len(FIMSet[0])):
                def eq_fim(m):
                    return m.TotalFIM[i,j] == m.my_block.egb.inputs['ele_'+str(i)+'_'+str(j)]
                
                con_name = "con"+str(i)+str(j)
                m.add_component(con_name, pyo.Constraint(expr=eq_fim))

        # add objective to maximize log det
        m.Obj = pyo.Objective(expr=m.my_block.egb.outputs['log_det'], sense=pyo.maximize)

    
    return m 

In [100]:
mip_option = True
objective = "A"
sparse_opt = True
fix_opt = False

each_manual_num = 5
total_manual_num = 20
budget_opt = 8000 

discretize_time = 10
num_dynamic_time = np.linspace(2,220,dynamic_Nt)
print(num_dynamic_time)

[  2.   4.   6.   8.  10.  12.  14.  16.  18.  20.  22.  24.  26.  28.
  30.  32.  34.  36.  38.  40.  42.  44.  46.  48.  50.  52.  54.  56.
  58.  60.  62.  64.  66.  68.  70.  72.  74.  76.  78.  80.  82.  84.
  86.  88.  90.  92.  94.  96.  98. 100. 102. 104. 106. 108. 110. 112.
 114. 116. 118. 120. 122. 124. 126. 128. 130. 132. 134. 136. 138. 140.
 142. 144. 146. 148. 150. 152. 154. 156. 158. 160. 162. 164. 166. 168.
 170. 172. 174. 176. 178. 180. 182. 184. 186. 188. 190. 192. 194. 196.
 198. 200. 202. 204. 206. 208. 210. 212. 214. 216. 218. 220.]


In [105]:
mod = compute_FIM(num_total, calculator.fim_collection, cost, mixed_integer=mip_option, 
                  num_dynamic_t = dynamic_Nt, num_dynamic_t_name = num_dynamic_time,
                  each_manual_number=each_manual_num, manual_number=total_manual_num,
                  dynamic_install_cost = install_cost_dynamic, discretize_time = discretize_time, 
                  obj=objective, fix=fix_opt, sparse=sparse_opt, budget=budget_opt)

if not mip_option and objective=="A":
    solver = pyo.SolverFactory('ipopt')
    solver.options['linear_solver'] = "ma57"
    solver.solve(mod, tee=True)


# if A, and MIP, use gurobi 
# relax mip gap a little (1%)
    
elif mip_option and objective=="A":
    solver = pyo.SolverFactory('mindtpy')
    results = solver.solve(mod, mip_solver='gurobi_persistent', nlp_solver='ipopt', 
                           time_limit=36000, iteration_limit=100000,
                           tee=True, integer_tolerance=0.5)

elif objective=="D":  
    solver = pyo.SolverFactory('cyipopt')
    solver.config.options['hessian_approximation'] = 'limited-memory' 
    additional_options={'max_iter':3000}
    
    for k,v in additional_options.items():
        solver.config.options[k] = v
    results = solver.solve(mod, tee=True)

{0: 2.0, 1: 4.0, 2: 6.0, 3: 8.0, 4: 10.0, 5: 12.0, 6: 14.0, 7: 16.0, 8: 18.0, 9: 20.0, 10: 22.0, 11: 24.0, 12: 26.0, 13: 28.0, 14: 30.0, 15: 32.0, 16: 34.0, 17: 36.0, 18: 38.0, 19: 40.0, 20: 42.0, 21: 44.0, 22: 46.0, 23: 48.0, 24: 50.0, 25: 52.0, 26: 54.0, 27: 56.0, 28: 58.0, 29: 60.0, 30: 62.0, 31: 64.0, 32: 66.0, 33: 68.0, 34: 70.0, 35: 72.0, 36: 74.0, 37: 76.0, 38: 78.0, 39: 80.0, 40: 82.0, 41: 84.0, 42: 86.0, 43: 88.0, 44: 90.0, 45: 92.0, 46: 94.0, 47: 96.0, 48: 98.0, 49: 100.0, 50: 102.0, 51: 104.0, 52: 106.0, 53: 108.0, 54: 110.0, 55: 112.0, 56: 114.0, 57: 116.0, 58: 118.0, 59: 120.0, 60: 122.0, 61: 124.0, 62: 126.0, 63: 128.0, 64: 130.0, 65: 132.0, 66: 134.0, 67: 136.0, 68: 138.0, 69: 140.0, 70: 142.0, 71: 144.0, 72: 146.0, 73: 148.0, 74: 150.0, 75: 152.0, 76: 154.0, 77: 156.0, 78: 158.0, 79: 160.0, 80: 162.0, 81: 164.0, 82: 166.0, 83: 168.0, 84: 170.0, 85: 172.0, 86: 174.0, 87: 176.0, 88: 178.0, 89: 180.0, 90: 182.0, 91: 184.0, 92: 186.0, 93: 188.0, 94: 190.0, 95: 192.0, 96: 19

In [106]:
### FIM
fim_result = np.zeros((5,5))
for i in range(5):
    for j in range(i,5):
        fim_result[i,j] = fim_result[j,i] = pyo.value(mod.TotalFIM[i,j])
        
print(fim_result)  
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))

print("Pyomo OF:", pyo.value(mod.Obj))
print("Log_det:", np.log(np.linalg.det(fim_result)))

[[ 5.20007838e+03  4.02505317e+01 -4.23398470e+03  1.35136882e+04
  -6.07019430e+02]
 [ 4.02505317e+01  2.32451406e+00 -3.08650628e+01  1.47691693e+02
  -2.48265746e-01]
 [-4.23398470e+03 -3.08650628e+01  3.51965553e+03 -1.13384291e+04
   4.41413564e+02]
 [ 1.35136882e+04  1.47691693e+02 -1.13384291e+04  3.90817200e+04
  -1.24733762e+03]
 [-6.07019430e+02 -2.48265746e-01  4.41413564e+02 -1.24733762e+03
   2.78945508e+02]]
trace: 48082.723924860446
det: 112214140433.83366
Pyomo OF: 48082.723924860446
Log_det: 25.44367485091609


In [107]:
# problem size calculator
if False:
    nr = num_total
    npa = 5
    nd = 5
    nd_time = 110
    num_var = (nr*nr+nr)/2 + (npa*npa+npa)/2 + nd
    num_equal = (npa*npa+npa)/2+2
    num_ineq = 1.5*(nr**2-nr)+2+nd+2*nd*nd_time

    print(num_var, num_equal, num_ineq)

In [108]:
ans_y = np.zeros((num_total,num_total))

pyomo_y = np.zeros((num_total, num_total))

for i in range(num_total):
    for j in range(i, num_total):
        cov = pyo.value(mod.cov_y[i,j])
        ans_y[i,j] = ans_y[j,i] = cov 
        
print("Cost for this stragy:", pyo.value(mod.cost))
print("Cost if no budget:", sum(cost))



#print(pyo.value(mod.cov_y[9,9]))
#print(pyo.value(mod.if_install_dynamic[9]))

#for i in range(9,119):
#    print(pyo.value(mod.cov_y[i,i]))
#print(sum(pyo.value(mod.cov_y[i,i]) for i in range(9,119)))

Cost for this stragy: 8000.000075789653
Cost if no budget: 63000


In [109]:
for i in range(len(ans_y)):
    for j in range(len(ans_y[0])):
        if ans_y[i][j] < 0.05:
            ans_y[i][j] = int(0)
        elif ans_y[i][j] > 0.95:
            ans_y[i][j] = int(1)
        else: 
            ans_y[i][j] = round(ans_y[i][j], 2)
print(ans_y)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [110]:
import pickle

store = True

if store:
    file = open('Mar15_A_110_8000_mip', 'wb')

    pickle.dump(ans_y, file)

    file.close()

In [111]:
for c in range(num_static):
    print(static_name[c], ": ", ans_y[c,c])

sol_y = np.asarray([ans_y[i,i] for i in range(num_static, num_total)])

sol_y = np.reshape(sol_y, (num_dynamic, dynamic_Nt))
np.around(sol_y)

for r in range(len(sol_y)):
    print(dynamic_name[r], ": ", sol_y[r])
    
# verify cost 
real_cost = 0 
for i in range(num_total):
    real_cost += ans_y[i,i]*cost[i]
    
dynamic_inst = [0]*num_dynamic
for j in range(num_dynamic):
    for t in range(dynamic_Nt):
        if sol_y[j,t]>0.99:
            dynamic_inst[j] = 1 
            
real_cost += sum(dynamic_inst[i]*install_cost_dynamic[i] for i in range(num_dynamic))
print(dynamic_inst)
print(real_cost)
    


Ads.gas_inlet.F :  1.0
Ads.gas_outlet.F :  0.0
Ads.gas_outlet.T :  0.0
Des.gas_inlet.F :  1.0
Des.gas_outlet.F :  1.0
Des.gas_outlet.T :  1.0
Ads.T_g.Value(19,10) :  1.0
Ads.T_g.Value(23,10) :  0.0
Ads.T_g.Value(28,10) :  1.0
Ads.gas_outlet.z("CO2") :  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Des.gas_outlet.z("CO2") :  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Ads.z("CO2",19,10) :  [0. 0. 0. 0. 0. 0. 0. 0. 0

In [ ]:
import matplotlib

def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw=None, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)
        
        
    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
#measure_name = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']
#measure_name = ['Ads.gas_outlet.z("CO2")', 'Des.gas_outlet.z("CO2")', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']
measure_name = ['a', 'b', 'c', 'd', 'e']

#time_set = ['500', '1000', '1500', '2000', '2200']
time_set = np.linspace(20, 2200, number_t)
time_set = [str(i) for i in time_set]
print(len(time_set), len(measure_name))
print(np.shape(sol_y))

fig, ax = plt.subplots(figsize=(16,8))

im, cbar = heatmap(sol_y, time_set, measure_name, ax=ax,
                   cmap="YlGn", cbarlabel="weight")
texts = annotate_heatmap(im, valfmt="{x:.1f}")

fig.tight_layout()
plt.show()

In [ ]:
def plot_data(budget, a_cri, d_cri):

    si=18
    plt.rc('axes', titlesize=si)
    plt.rc('axes', labelsize=si)
    plt.rc('xtick', labelsize=si)
    plt.rc('ytick', labelsize=si)

    plt.scatter(budget, np.log10(a_cri), s=50, color='red', label='A-optimality')
    #plt.scatter(budget, np.log10(d_cri), marker='v', s=50, color='blue', label='D-optimality')
    plt.xlabel('Budget', fontweight="bold")
    plt.ylabel('log10(A-optimality)', fontweight="bold")
    
    plt.show()
    
    
    si=18
    plt.rc('axes', titlesize=si)
    plt.rc('axes', labelsize=si)
    plt.rc('xtick', labelsize=si)
    plt.rc('ytick', labelsize=si)

    #plt.scatter(budget, np.log10(a_cri), s=50, color='red', label='A-optimality')
    plt.scatter(budget, np.log10(d_cri), marker='v', s=50, color='blue', label='D-optimality')
    plt.xlabel('Budget', fontweight="bold")
    plt.ylabel('log10(D-optimality)', fontweight="bold")
    
    plt.show()

In [ ]:
a_set = [65693.94, 66574.63, 67474.485, 67608.817]
d_set = [np.exp(30.13), np.exp(30.719), np.exp(31.1205), np.exp(31.2586)]

plot_data([3000,4000,5000,6400], a_set, d_set)